[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). This notebook is released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).

# DIY Covid-19 Dashboard

This is a dashboard displaying the data related to Covid-19 cases in England.
All the data is from [Public Health England](https://www.gov.uk/government/organisations/public-health-england) (PHE) which is one of many Institutions worldwide running a Coronavirus [dashboard](https://coronavirus.data.gov.uk/), with current statistics on the pandemic. 

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [3]:
# Load JSON files and store the raw data in some variable. 
with open("sexDeathDistribution.json","rt") as file:
    data=json.load(file)
datadic= data['data'][0]
jsondata=datadic

In [10]:
# Wrangle the data into a DataFrame, put it as a function.
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Wrangle the data, create a dataframe"""
   
    females = jsondata['females']
    males = jsondata['males']
    
    ageranges = [c['age'] for c in females]
    
    def min_age(agerange):
        agerange=agerange.replace('+','')
        start=agerange.split('_')[0]
        return int(start)
    ageranges.sort(key=min_age)

    age_df=pd.DataFrame(index=ageranges,columns=['females','males','total'])

    for entry in females:
        ageband=entry['age']
        age_df.loc[ageband,'females']=entry['value']

    for entry in males:
        ageband=entry['age']
        age_df.loc[ageband,'males']=entry['value']

    age_df['total']=age_df['females']+age_df['males']
    return age_df

df=wrangle_data(jsondata) # df is the dataframe for plotting

In [11]:
# Access the API which will be called by the button callback.
def access_api():
    """ Accesses the PHE API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    #Retrieve the data of death within 28 days for female and male
    filters = [
        'areaType=nation',
        'areaName=England'
    ]
    structure={
        "females": "femaleDeaths28Days",
        "males" : "maleDeaths28Days"
    }
    api = Cov19API(filters=filters, structure=structure)
    sexDeathDistribution=api.get_json()
    return sexDeathDistribution # return data read from the API

## Refresh Button
Click the button below to refresh the data.


In [19]:
def api_button_callback(button):
    """ Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    refresh_graph()
    apibutton.icon="check"
    apibutton.disabled=True
    
apibutton=wdg.Button(
    description='Refresh', 
    disabled=False,
    button_style='info',
    tooltip="Refresh the data",
    icon='download'
)

#Change the style of the button
apibutton.style.button_color = 'green'  # Change button color
apibutton.style.font_weight = 'bold'  # Make the font bold

#Register button callback function with the button
apibutton.on_click(api_button_callback)

#Display the button
display(apibutton)

Button(button_style='info', description='Refresh', icon='download', style=ButtonStyle(button_color='green', fo…

## Graphs and Analysis

Below is the graph showing the **number of death within 28 days of a positive test for females and males from different ages in England**.
You can choose from the selection of sex to see the graph of females and males and total numbers.

In [13]:
def age_graph(graphcolumns):
    ncols=len(graphcolumns)
    if ncols>0:
        df.plot(kind='bar', y=list(graphcolumns))
        plt.show()
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")

agecols=wdg.SelectMultiple(
    options=('females','males','total'),
    value=('females','males'),
    rows=3,
    description='Sex',
    disabled=False
)

def refresh_graph():
    """ Change the value of the widget in order to force a redraw of the graph"""
    current=agecols.value
    agecols.value=[]
    agecols.value=current
    output=wdg.interactive_output(age_graph,{'graphcolumns':agecols})
    
# connect the plotting function and the widget
output=wdg.interactive_output(age_graph,{'graphcolumns':agecols})

# actually displays the graph
display(agecols,output)

SelectMultiple(description='Sex', index=(0, 1), options=('females', 'males', 'total'), rows=3, value=('females…

Output()

**Author and Copyright Notice** Remember that if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge your sources: Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england) and on the [DIY Covid Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash), Copyright (C) Fabrizio Smeraldi 2020,2023. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).